# PRÁCTICA: Mejorando la Relevancia en un Sistema RAG

## 1. Preparación del entorno

Instalamos las librerías necesarias:

In [ ]:
# pip install chromadb sentence-transformers


Importamos los módulos principales:

## 2. Crear una mini base de conocimiento

Vamos a simular una base de datos con textos sobre tecnología, astronomía y animales.

In [7]:
base_conocimiento = {
    "Tecnología": [
        "La inteligencia artificial (IA) es un campo de la informática que se enfoca en la creación de sistemas que pueden razonar, aprender y actuar de manera inteligente.",
        "Python es un lenguaje de programación de alto nivel muy popular para el desarrollo web, el análisis de datos y el aprendizaje automático.",
        "Blockchain es una tecnología de registro distribuido que permite almacenar datos de manera segura y transparente.",
        "El Internet de las Cosas (IoT) conecta objetos cotidianos a internet, permitiendo la comunicación y el control remoto."
    ],
    "Astronomía": [
        "La Vía Láctea es la galaxia espiral donde se encuentra nuestro Sistema Solar.",
        "Un agujero negro es una región del espacio de la que nada, ni siquiera la luz, puede escapar debido a la inmensa gravedad.",
        "Marte, el planeta rojo, es un objetivo clave para la exploración espacial debido a la posibilidad de albergar vida pasada o futura.",
        "Una supernova es una explosión estelar extremadamente brillante y poderosa."
    ],
    "Animales": [
        "El guepardo es el animal terrestre más rápido, capaz de alcanzar velocidades de hasta 120 km/h en ráfagas cortas.",
        "Los pulpos son conocidos por su alta inteligencia, capacidad de camuflaje y tener tres corazones.",
        "La migración anual del ñu en el Serengeti es uno de los mayores espectáculos de la vida salvaje en la Tierra.",
        "Las abejas desempeñan un papel vital en la polinización de muchas especies de plantas, incluyendo gran parte de los cultivos."
    ]
}


In [8]:
def mostrar_categorias(db):
    """Muestra todas las categorías disponibles en la base de conocimiento."""
    print("--- 📂 Categorías Disponibles ---")
    for categoria in db.keys():
        print(f"- **{categoria}**")
    print("-" * 30)


In [9]:
def obtener_datos_por_categoria(db, categoria):
    """
    Retorna los textos asociados a una categoría específica.
    Retorna una lista vacía si la categoría no existe.
    """
    return db.get(categoria, [])


In [10]:
mostrar_categorias(base_conocimiento)

--- 📂 Categorías Disponibles ---
- **Tecnología**
- **Astronomía**
- **Animales**
------------------------------


In [11]:
categoria_buscada = "Tecnología"
datos_tecnologia = obtener_datos_por_categoria(base_conocimiento, categoria_buscada)

In [12]:
print(f"--- 💡 Textos sobre {categoria_buscada} ---")
if datos_tecnologia:
    for i, texto in enumerate(datos_tecnologia, 1):
        print(f"**{i}.** {texto}")
else:
    print(f"No se encontraron datos para la categoría '{categoria_buscada}'.")
print("-" * 30)

# 3. Buscar datos en otra categoría (ejemplo: Animales)
categoria_animales = "Animales"
datos_animales = obtener_datos_por_categoria(base_conocimiento, categoria_animales)

print(f"--- 🐾 Textos sobre {categoria_animales} ---")
for i, texto in enumerate(datos_animales, 1):
    print(f"**{i}.** {texto}")
print("-" * 30)

--- 💡 Textos sobre Tecnología ---
**1.** La inteligencia artificial (IA) es un campo de la informática que se enfoca en la creación de sistemas que pueden razonar, aprender y actuar de manera inteligente.
**2.** Python es un lenguaje de programación de alto nivel muy popular para el desarrollo web, el análisis de datos y el aprendizaje automático.
**3.** Blockchain es una tecnología de registro distribuido que permite almacenar datos de manera segura y transparente.
**4.** El Internet de las Cosas (IoT) conecta objetos cotidianos a internet, permitiendo la comunicación y el control remoto.
------------------------------
--- 🐾 Textos sobre Animales ---
**1.** El guepardo es el animal terrestre más rápido, capaz de alcanzar velocidades de hasta 120 km/h en ráfagas cortas.
**2.** Los pulpos son conocidos por su alta inteligencia, capacidad de camuflaje y tener tres corazones.
**3.** La migración anual del ñu en el Serengeti es uno de los mayores espectáculos de la vida salvaje en la Tierr

## 3. Crear la base vectorial (ChromaDB)

Creamos una colección donde almacenaremos los embeddings de los textos.

In [13]:
import chromadb
from chromadb.utils import embedding_functions

In [14]:
# 1. Configurar el cliente ChromaDB
# Usaremos un cliente persistente, lo que significa que guardará los datos
# en un directorio local para que persistan entre sesiones.
CHROMA_PATH = "chroma_db_tech"
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

print(f"Base de datos ChromaDB configurada en el directorio: {CHROMA_PATH}")

Base de datos ChromaDB configurada en el directorio: chroma_db_tech


In [15]:
# 2. Configurar el Modelo de Embeddings
# Usaremos un modelo de Hugging Face de la librería 'sentence-transformers'
# que es excelente para tareas de embeddings de texto.
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
hf_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL
)

c:\Users\X415\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [16]:
# 3. Crear la Colección
# Una "colección" en ChromaDB es donde se almacenan los textos (documentos)
# y sus representaciones vectoriales (embeddings).
COLLECTION_NAME = "base_conocimiento_tecnologia_astronomia_animales"
collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=hf_ef,
    metadata={"hnsw:space": "cosine"} # Define la métrica de distancia
)

print(f"Colección '{COLLECTION_NAME}' creada (o cargada si ya existía).")

Colección 'base_conocimiento_tecnologia_astronomia_animales' creada (o cargada si ya existía).


In [17]:
# 4. Preparar los datos de la mini base de conocimiento (del punto anterior)

# Aplanamos los datos para que sean más fáciles de ingresar a ChromaDB.
# También creamos IDs únicos y metadatos (la categoría original).
documentos = []
metadatos = []
ids = []
id_counter = 1


for categoria, textos in base_conocimiento.items():
    for texto in textos:
        documentos.append(texto)
        metadatos.append({"categoria": categoria})
        ids.append(f"doc_{id_counter}")
        id_counter += 1


In [18]:

# 5. Agregar los documentos a la Colección (Cálculo de Embeddings)
# Chroma calcula automáticamente los embeddings de los 'documentos' usando 'hf_ef'.
print("\nIniciando la inserción de documentos y cálculo de embeddings...")
collection.add(
    documents=documentos,
    metadatas=metadatos,
    ids=ids
)
print(f"¡Éxito! Se insertaron {len(documentos)} documentos en la base vectorial.")

print(f"Documentos totales en la colección: {collection.count()}")


Iniciando la inserción de documentos y cálculo de embeddings...
¡Éxito! Se insertaron 12 documentos en la base vectorial.
Documentos totales en la colección: 12


## 4. Realizar una búsqueda semántica básica

Consultamos algo relacionado con animales domésticos:

In [19]:
# 1. Definir la consulta (el texto de entrada)
pregunta_usuario = "¿Qué información tienes sobre animales domésticos o mascotas?"

print(f"\n--- 🧠 Realizando Búsqueda Semántica ---")
print(f"Pregunta: **{pregunta_usuario}**")


--- 🧠 Realizando Búsqueda Semántica ---
Pregunta: **¿Qué información tienes sobre animales domésticos o mascotas?**


In [20]:
# 2. Realizar la consulta en la colección
# ChromaDB convertirá automáticamente la pregunta_usuario en un vector
# y buscará los vectores más similares dentro de la colección.
resultados = collection.query(
    query_texts=[pregunta_usuario], # La pregunta que queremos vectorizar y buscar
    n_results=2,                    # Queremos los 2 resultados más relevantes
    # Puedes añadir where={} para filtrar por metadatos, por ejemplo:
    # where={"categoria": "Animales"}
)


In [21]:
# 3. Procesar y mostrar los resultados
print("\n--- 🎯 Resultados de la Búsqueda ---")

# Los resultados vienen en formato de lista de listas (aunque solo consultamos una vez)
# Usaremos el primer elemento [0] de cada lista.
documentos_relevantes = resultados['documents'][0]
distancias = resultados['distances'][0]
metadatos_relevantes = resultados['metadatas'][0]

if documentos_relevantes:
    for i, (doc, meta, dist) in enumerate(zip(documentos_relevantes, metadatos_relevantes, distancias)):
        # La distancia es una medida de qué tan lejos está el embedding del documento
        # del embedding de la pregunta. Una distancia menor indica mayor similitud.

        print(f"\n**{i + 1}. Documento Relevante** (Distancia: {dist:.4f})")
        print(f"   -> **Categoría:** {meta['categoria']}")
        print(f"   -> **Texto:** {doc}")
else:
    print("No se encontraron documentos relevantes.")

print("-" * 40)


--- 🎯 Resultados de la Búsqueda ---

**1. Documento Relevante** (Distancia: 0.4664)
   -> **Categoría:** Animales
   -> **Texto:** El guepardo es el animal terrestre más rápido, capaz de alcanzar velocidades de hasta 120 km/h en ráfagas cortas.

**2. Documento Relevante** (Distancia: 0.5237)
   -> **Categoría:** Animales
   -> **Texto:** Los pulpos son conocidos por su alta inteligencia, capacidad de camuflaje y tener tres corazones.
----------------------------------------


## 5. Optimización 1: Chunking Estratégico

Supongamos que tenemos textos largos. Vamos a dividirlos en chunks lógicos con solapamiento (simulado).

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

texto_largo_ejemplo = """
La inteligencia artificial (IA) es un campo de la informática que se enfoca en la creación de sistemas que pueden razonar, aprender y actuar de manera inteligente. Los modelos de lenguaje grande (LLMs) como GPT-4 han revolucionado la interacción humana con la tecnología. Estos modelos son entrenados en vastos conjuntos de datos para predecir la siguiente palabra, permitiendo generar texto coherente y relevante.

Sin embargo, el universo es mucho más que bits y bytes. La Vía Láctea es la galaxia espiral donde se encuentra nuestro Sistema Solar, conteniendo miles de millones de estrellas. En su centro reside un agujero negro supermasivo, Sagitario A*, cuya inmensa gravedad dicta el movimiento de todo lo que le rodea. Estudiar estos fenómenos requiere telescopios potentes y análisis de datos avanzados.

En la Tierra, la biología nos ofrece maravillas como el pulpo, un animal conocido por su alta inteligencia. Los pulpos son maestros del camuflaje, capaces de cambiar el color y la textura de su piel en un instante para mimetizarse con el entorno. Además, poseen un sistema circulatorio único con tres corazones. Su capacidad de resolver problemas los hace fascinantes para la neurociencia.
"""


In [23]:
# 2. Configurar el Text Splitter
# Usaremos un RecursiveCharacterTextSplitter que intenta dividir por diferentes
# caracteres (saltos de línea, puntos, etc.) para mantener el contexto.
text_splitter = RecursiveCharacterTextSplitter(
    # Tamaño máximo de cada chunk
    chunk_size=300,
    # Tamaño del solapamiento entre chunks adyacentes
    chunk_overlap=50,
    # Separadores preferidos (los predeterminados son buenos)
    separators=["\n\n", "\n", ".", " ", ""]
)


In [24]:
# 3. Aplicar el Chunking
# La función .create_documents toma el texto y lo divide según la configuración.
chunks_documentos = text_splitter.create_documents([texto_largo_ejemplo])

print(f"\n--- ✂️ Resultado del Chunking Estratégico ---")
print(f"Texto original dividido en {len(chunks_documentos)} chunks.")


--- ✂️ Resultado del Chunking Estratégico ---
Texto original dividido en 6 chunks.


In [25]:
# 4. Preparar la nueva estructura de datos para ChromaDB
nuevos_documentos = []
nuevos_metadatos = []
nuevos_ids = []
id_base = 100 # Usamos IDs diferentes para distinguirlos

print("\nChunks generados (ejemplo):")
for i, chunk in enumerate(chunks_documentos):
    chunk_text = chunk.page_content
    # Simulamos asignar una categoría basada en el contenido inicial (no ideal, pero simple)
    if "IA" in chunk_text or "GPT-4" in chunk_text:
        categoria = "Tecnología_Chunked"
    elif "Vía Láctea" in chunk_text or "agujero negro" in chunk_text:
        categoria = "Astronomía_Chunked"
    elif "pulpo" in chunk_text or "biología" in chunk_text:
        categoria = "Animales_Chunked"
    else:
        categoria = "General_Chunked"

    nuevos_documentos.append(chunk_text)
    nuevos_metadatos.append({
        "categoria": categoria,
        "source": "Artículo Largo",
        "chunk_id": i
    })
    nuevos_ids.append(f"chunk_{id_base + i}")
    print(f"**Chunk {i}:** (Solapamiento) {chunk_text[:70]}...") # Mostramos solo el inicio


Chunks generados (ejemplo):
**Chunk 0:** (Solapamiento) La inteligencia artificial (IA) es un campo de la informática que se e...
**Chunk 1:** (Solapamiento) . Estos modelos son entrenados en vastos conjuntos de datos para prede...
**Chunk 2:** (Solapamiento) Sin embargo, el universo es mucho más que bits y bytes. La Vía Láctea ...
**Chunk 3:** (Solapamiento) . En su centro reside un agujero negro supermasivo, Sagitario A*, cuya...
**Chunk 4:** (Solapamiento) En la Tierra, la biología nos ofrece maravillas como el pulpo, un anim...
**Chunk 5:** (Solapamiento) . Además, poseen un sistema circulatorio único con tres corazones. Su ...


In [26]:
# 5. Sobreescribir o añadir a la Colección (Re-indexación)

# NOTA IMPORTANTE: Para este ejemplo, vamos a *limpiar* la colección
# para que solo contenga los chunks, o usar una nueva.
# Aquí simplemente añadiremos los nuevos chunks:
# collection.add(...)

# Para asegurarnos de no mezclar datos si el script se ejecuta muchas veces,
# vamos a usar un cliente nuevo y una colección temporal para este chunk:
CHROMA_PATH_CHUNKS = "chroma_db_chunked"
chroma_client_chunked = chromadb.PersistentClient(path=CHROMA_PATH_CHUNKS)
COLLECTION_NAME_CHUNKS = "base_conocimiento_chunked"

# Creamos o cargamos la nueva colección con los embeddings redefinidos (hf_ef debe estar definido)
collection_chunked = chroma_client_chunked.get_or_create_collection(
    name=COLLECTION_NAME_CHUNKS,
    embedding_function=hf_ef, # Asume que hf_ef está definido del paso 3
    metadata={"hnsw:space": "cosine"}
)

print("\nIniciando la inserción de CHUNKS y cálculo de embeddings...")
collection_chunked.add(
    documents=nuevos_documentos,
    metadatas=nuevos_metadatos,
    ids=nuevos_ids
)

print(f"¡Éxito! Se insertaron {collection_chunked.count()} CHUNKS en la nueva base vectorial.")
print("-" * 50)


Iniciando la inserción de CHUNKS y cálculo de embeddings...
¡Éxito! Se insertaron 6 CHUNKS en la nueva base vectorial.
--------------------------------------------------


## 6. Optimización 2: Re-ranking

Vamos a mejorar la búsqueda semántica seleccionando los resultados más relevantes mediante una segunda evaluación (re-ranking).

In [28]:
from sentence_transformers import CrossEncoder

# 1. Cargar el modelo Re-ranker (Cross-Encoder)
# Los Cross-Encoders son modelos que toman un par de (consulta, documento)
# y devuelven una puntuación de similitud. Son más lentos que los bi-encoders
# pero mucho más precisos para clasificar la relevancia.
RERANKER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
cross_encoder = CrossEncoder(RERANKER_MODEL)

In [30]:

print(f"\n--- 🔄 Modelo Cross-Encoder cargado: {RERANKER_MODEL} ---")

# 2. Definir la Consulta
pregunta_usuario_rerank = "¿Qué información tienes sobre el comportamiento de los animales acuáticos?"

# 3. Paso 1: Recuperación Inicial (Usando la colección con Chunks)
# Asumimos que 'collection_chunked' está cargada del paso 5.
# Recuperamos un número mayor de resultados (ej. 5) para tener un buen pool para re-rankear.
NUM_RESULTS_RETRIEVED = 5
print(f"Buscando los {NUM_RESULTS_RETRIEVED} chunks iniciales más cercanos...")

resultados_iniciales = collection_chunked.query(
    query_texts=[pregunta_usuario_rerank],
    n_results=NUM_RESULTS_RETRIEVED,
)

documentos_recuperados = resultados_iniciales['documents'][0]
metadatos_recuperados = resultados_iniciales['metadatas'][0]

# 4. Paso 2: Preparar los pares (Consulta, Documento) para el Cross-Encoder
# El Cross-Encoder necesita una lista de tuplas: [(pregunta, chunk_1), (pregunta, chunk_2), ...]
pares_para_rerank = [
    (pregunta_usuario_rerank, chunk) for chunk in documentos_recuperados
]

# 5. Paso 3: Obtener las Puntuaciones de Relevancia del Cross-Encoder
# El método .predict() devuelve una puntuación para cada par. Una puntuación más alta es mejor.
puntuaciones = cross_encoder.predict(pares_para_rerank)

# 6. Paso 4: Combinar Resultados y Reordenar
# Creamos una lista de diccionarios que incluye el chunk, metadato y la nueva puntuación.
resultados_combinados = []
for doc, meta, score in zip(documentos_recuperados, metadatos_recuperados, puntuaciones):
    resultados_combinados.append({
        'document': doc,
        'metadata': meta,
        'rerank_score': score
    })

# Ordenar la lista por la puntuación del re-ranker (de mayor a menor)
resultados_ordenados = sorted(
    resultados_combinados,
    key=lambda x: x['rerank_score'],
    reverse=True
)

# 7. Mostrar los resultados Re-rankeados (solo los top 2)
NUM_FINAL_RESULTS = 2
print(f"\n--- ✨ Resultados Finales Re-rankeados (Top {NUM_FINAL_RESULTS}) ---")

for i, resultado in enumerate(resultados_ordenados[:NUM_FINAL_RESULTS]):
    print(f"\n**{i + 1}. Documento Relevante** (Puntuación: {resultado['rerank_score']:.4f})")
    print(f"   -> **Categoría:** {resultado['metadata']['categoria']}")
    print(f"   -> **Texto:** {resultado['document']}")

print("-" * 50)


--- 🔄 Modelo Cross-Encoder cargado: cross-encoder/ms-marco-MiniLM-L-6-v2 ---
Buscando los 5 chunks iniciales más cercanos...

--- ✨ Resultados Finales Re-rankeados (Top 2) ---

**1. Documento Relevante** (Puntuación: -4.3383)
   -> **Categoría:** Animales_Chunked
   -> **Texto:** En la Tierra, la biología nos ofrece maravillas como el pulpo, un animal conocido por su alta inteligencia. Los pulpos son maestros del camuflaje, capaces de cambiar el color y la textura de su piel en un instante para mimetizarse con el entorno

**2. Documento Relevante** (Puntuación: -8.9486)
   -> **Categoría:** Astronomía_Chunked
   -> **Texto:** . En su centro reside un agujero negro supermasivo, Sagitario A*, cuya inmensa gravedad dicta el movimiento de todo lo que le rodea. Estudiar estos fenómenos requiere telescopios potentes y análisis de datos avanzados.
--------------------------------------------------


## 7. Optimización 3: Búsqueda híbrida

Combinaremos búsqueda por palabra clave + semántica.

In [31]:
def simulacion_sparse_retrieval(pregunta, documentos_chunked):
    """
    Simula la búsqueda por palabra clave (BM25/Sparse) buscando cualquier
    documento cuyo texto contenga al menos una palabra clave de la pregunta.
    """
    palabras_clave = set(pregunta.lower().split())
    resultados_sparse = []
    
    for i, doc_data in enumerate(documentos_chunked):
        texto = doc_data['document'].lower()
 
        if any(keyword in texto for keyword in palabras_clave if len(keyword) > 3):
             resultados_sparse.append({
                'id': doc_data['id'],
                'document': doc_data['document'],
                'metadata': doc_data['metadata']
            })

    return resultados_sparse[:5]

In [32]:
all_chroma_data = collection_chunked.get(include=['metadatas', 'documents'])
all_chunks_data = [
    {'id': all_chroma_data['ids'][i], 
     'document': all_chroma_data['documents'][i], 
     'metadata': all_chroma_data['metadatas'][i]} 
    for i in range(len(all_chroma_data['ids']))
]

# 2. Definir la Consulta y realizar ambas búsquedas
pregunta_hibrida = "Dame información sobre el agujero negro en el centro de nuestra galaxia."

# A. Búsqueda Semántica (Dense)
print(f"\n--- 🔎 Búsqueda Semántica ---")
# Recuperamos 10 resultados para el pool
resultados_dense = collection_chunked.query(
    query_texts=[pregunta_hibrida],
    n_results=10,
    include=['documents', 'metadatas', 'distances']
)

# B. Búsqueda por Palabra Clave (Sparse Simulation)
print(f"--- 🔑 Búsqueda por Palabra Clave ---")
resultados_sparse = simulacion_sparse_retrieval(pregunta_hibrida, all_chunks_data)
print(f"Palabra clave recuperó {len(resultados_sparse)} chunks.")


# 3. Aplicar Reciprocal Rank Fusion (RRF)
# Función RRF para combinar y reordenar las dos listas de resultados
def rank_fusion(dense_results, sparse_results, k=60):
    """Implementa Reciprocal Rank Fusion (RRF) para combinar rankings."""
    fused_scores = {}
    
    # A. Procesar resultados Semánticos (Dense)
    # dense_results es el output de ChromaDB query
    ids_dense = dense_results['ids'][0]
    for rank, doc_id in enumerate(ids_dense, 1):
        score = 1 / (rank + k)
        fused_scores[doc_id] = fused_scores.get(doc_id, 0) + score

    # B. Procesar resultados Palabra Clave (Sparse)
    # sparse_results es una lista de diccionarios que simula la respuesta sparse
    for rank, result in enumerate(sparse_results, 1):
        doc_id = result['id']
        score = 1 / (rank + k)
        fused_scores[doc_id] = fused_scores.get(doc_id, 0) + score
        
    # C. Obtener el ranking final basado en la puntuación RRF
    ranked_ids = sorted(fused_scores, key=fused_scores.get, reverse=True)
    
    # Recuperar los detalles completos de los chunks (simulando una llamada a la DB)
    final_results = []
    
    # Mapear IDs a documentos completos para la salida final
    id_to_doc = {d['id']: d for d in all_chunks_data}
    
    for doc_id in ranked_ids:
        if doc_id in id_to_doc:
            final_results.append({
                'document': id_to_doc[doc_id]['document'],
                'metadata': id_to_doc[doc_id]['metadata'],
                'rrf_score': fused_scores[doc_id]
            })
            
    return final_results

# 4. Ejecutar y mostrar la Búsqueda Híbrida
resultados_hibridos = rank_fusion(resultados_dense, resultados_sparse)

print("\n--- 🌐 Resultados Finales Híbridos (RRF) ---")
# Mostramos los 3 resultados más relevantes
for i, resultado in enumerate(resultados_hibridos[:3]):
    print(f"\n**{i + 1}. Documento Híbrido** (RRF Score: {resultado['rrf_score']:.4f})")
    print(f"   -> **Categoría:** {resultado['metadata']['categoria']}")
    print(f"   -> **Texto:** {resultado['document'][:100]}...")

print("-" * 60)

Number of requested results 10 is greater than number of elements in index 6, updating n_results = 6



--- 🔎 Búsqueda Semántica ---
--- 🔑 Búsqueda por Palabra Clave ---
Palabra clave recuperó 1 chunks.

--- 🌐 Resultados Finales Híbridos (RRF) ---

**1. Documento Híbrido** (RRF Score: 0.0328)
   -> **Categoría:** Astronomía_Chunked
   -> **Texto:** . En su centro reside un agujero negro supermasivo, Sagitario A*, cuya inmensa gravedad dicta el mov...

**2. Documento Híbrido** (RRF Score: 0.0161)
   -> **Categoría:** Astronomía_Chunked
   -> **Texto:** Sin embargo, el universo es mucho más que bits y bytes. La Vía Láctea es la galaxia espiral donde se...

**3. Documento Híbrido** (RRF Score: 0.0159)
   -> **Categoría:** Animales_Chunked
   -> **Texto:** En la Tierra, la biología nos ofrece maravillas como el pulpo, un animal conocido por su alta inteli...
------------------------------------------------------------


## Extensión 

Implementa un caché simple que guarde los últimos resultados de consulta.
Tip: Usa un diccionario {consulta: resultados} y revisa si la consulta ya existe antes de buscar otra vez.

In [34]:
class QueryCache:
    """
    Un caché simple que almacena los resultados de búsquedas anteriores
    para evitar procesamiento redundante.
    """
    def __init__(self, max_size=5):
        # El diccionario para almacenar los resultados: {consulta_str: resultados}
        self.cache = {}
        # Límite de entradas para mantener el caché gestionable (LRU simple)
        self.max_size = max_size
        print(f"✅ Caché inicializado con tamaño máximo: {self.max_size}")

    def get_results(self, query):
        """
        Retorna los resultados si la consulta existe en el caché.
        Mueve la consulta al final (simulación LRU) para mantenerla fresca.
        """
        if query in self.cache:
            # Simulación simple de "usado recientemente": borra y reinserta
            # para moverlo al "final" del orden de inserción (mantenerlo fresco).
            results = self.cache.pop(query)
            self.cache[query] = results
            print(f"➡️ CACHÉ HIT: Resultados devueltos para la consulta: '{query[:30]}...'")
            return results
        
        print(f"❌ CACHÉ MISS: La consulta no está en el caché.")
        return None

    def store_results(self, query, results):
        """
        Almacena los resultados de una nueva consulta. Si el caché está lleno,
        elimina la entrada más antigua (el primer elemento insertado, simple LRU).
        """
        # 1. Chequeo de Límite (Simulación de "Least Recently Used - LRU")
        if len(self.cache) >= self.max_size:
            # Elimina el primer elemento (el más antiguo/menos recientemente usado)
            oldest_key = next(iter(self.cache))
            self.cache.pop(oldest_key)
            print(f"🗑️ Caché lleno. Eliminada la entrada más antigua: '{oldest_key[:30]}...'")

        # 2. Almacenar el nuevo resultado
        self.cache[query] = results
        print(f"➕ Resultado almacenado en caché para: '{query[:30]}...'")

    def clear(self):
        """Vacía completamente el caché."""
        self.cache = {}
        print("Caché limpiado.")

In [36]:
query_cache = QueryCache(max_size=3)


def buscar_hibrida_optimizada(query, cache_instance):
    """
    Simula la función de búsqueda híbrida (lenta) que utiliza el caché.
    """
    # 1. Revisar Caché
    cached_results = cache_instance.get_results(query)
    if cached_results is not None:
        return cached_results

    # 2. Si hay CACHÉ MISS (Simular la búsqueda lenta)
    import time
    print("...Ejecutando búsqueda semántica, sparse y RRF (simulación de 2s)...")
    time.sleep(2)  # Simula el tiempo que tomaría la búsqueda real y el re-ranking
    
    # Simular los resultados finales (lista de diccionarios)
    simulated_results = [
        {"document": f"Chunk relevante para '{query}'...", "score": 0.95},
        {"document": f"Chunk secundario para '{query}'...", "score": 0.88},
    ]

    # 3. Almacenar resultados
    cache_instance.store_results(query, simulated_results)
    return simulated_results

# --- Escenario 1: Nueva Consulta (CACHÉ MISS) ---
consulta_A = "¿Qué información tienes sobre el agujero negro en el centro de nuestra galaxia?"
print("\n--- Intento 1 (Consulta A) ---")
resultados_A = buscar_hibrida_optimizada(consulta_A, query_cache)
# print(resultados_A)

# --- Escenario 2: Repetición Inmediata (CACHÉ HIT) ---
print("\n--- Intento 2 (Consulta A - Repetida) ---")
resultados_A_repetido = buscar_hibrida_optimizada(consulta_A, query_cache)
# ¡Tiempo de espera evitado!

# --- Escenario 3: Nueva Consulta (CACHÉ MISS) ---
consulta_B = "¿Cuál es el animal terrestre más rápido del mundo?"
print("\n--- Intento 3 (Consulta B) ---")
resultados_B = buscar_hibrida_optimizada(consulta_B, query_cache)

# --- Escenario 4: Nueva Consulta que llena el caché y elimina el más antiguo ---
consulta_C = "Menciona los beneficios del blockchain."
print("\n--- Intento 4 (Consulta C - Límite de Caché) ---")
resultados_C = buscar_hibrida_optimizada(consulta_C, query_cache)
# La Consulta A fue eliminada porque era la más antigua y el caché solo permite 3 entradas.

# --- Escenario 5: Volver a consultar A (Ahora será CACHÉ MISS) ---
print("\n--- Intento 5 (Consulta A - Eliminada) ---")
resultados_A_re_miss = buscar_hibrida_optimizada(consulta_A, query_cache)
# ¡Vuelve a ejecutar la búsqueda lenta!

✅ Caché inicializado con tamaño máximo: 3

--- Intento 1 (Consulta A) ---
❌ CACHÉ MISS: La consulta no está en el caché.
...Ejecutando búsqueda semántica, sparse y RRF (simulación de 2s)...
➕ Resultado almacenado en caché para: '¿Qué información tienes sobre ...'

--- Intento 2 (Consulta A - Repetida) ---
➡️ CACHÉ HIT: Resultados devueltos para la consulta: '¿Qué información tienes sobre ...'

--- Intento 3 (Consulta B) ---
❌ CACHÉ MISS: La consulta no está en el caché.
...Ejecutando búsqueda semántica, sparse y RRF (simulación de 2s)...
➕ Resultado almacenado en caché para: '¿Cuál es el animal terrestre m...'

--- Intento 4 (Consulta C - Límite de Caché) ---
❌ CACHÉ MISS: La consulta no está en el caché.
...Ejecutando búsqueda semántica, sparse y RRF (simulación de 2s)...
➕ Resultado almacenado en caché para: 'Menciona los beneficios del bl...'

--- Intento 5 (Consulta A - Eliminada) ---
➡️ CACHÉ HIT: Resultados devueltos para la consulta: '¿Qué información tienes sobre ...'
